In [1]:
import numpy as np
import pandas as pd
from typing import List, Tuple
from itertools import combinations
import random


class TabuSearch:
    def __init__(self, n_iters: int, n_tasks: int, max_len_swap_tabu_list: int, max_len_candidate_tabu_list: int, neighborhood_sample_size: int = 5, greedy_criterion: str = "due_date"):        
        self.max_len_swap_tabu_list = max_len_swap_tabu_list
        self.max_len_candidate_tabu_list = max_len_candidate_tabu_list
        self.n_iters = n_iters
        self.n_tasks = n_tasks
        self.neighborhood_sample_size = neighborhood_sample_size

        self.tasks = self.generate_tasks()
        result_greedy =  self.greedy(tasks=self.tasks, criterion=greedy_criterion)  # search trajectory starting point
        self.current_best_solution = result_greedy["id"].to_list()
        self.swap_tabu_list = []
        if max_len_candidate_tabu_list > 0:
            self.candidate_tabu_list = [self.current_best_solution]
        else:
            self.candidate_tabu_list = []


    def generate_tasks(self) -> pd.DataFrame:
        due_date = np.random.randint(low=5, high=30, size=self.n_tasks)
        processing_time = np.maximum(1, due_date - np.random.randint(low=0, high=5, size=self.n_tasks))
        importance = np.random.randint(low=1, high=6, size=self.n_tasks)

        return pd.DataFrame({
            "id": range(self.n_tasks),
            "due_date": due_date,
            "processing_time": processing_time,
            "importance": importance,
            "done": False,
            "tardiness": 0,
            "execution_order": 0,
            "completion_time": 0})


    def greedy(self, tasks: pd.DataFrame, criterion: str) -> pd.DataFrame:
        if criterion == "SPT":  # shortest processing time; optimal solution to minimize elapsed waiting time 
            sort_prio = ["processing_time", "due_date"]
        elif criterion == "EDD":  # earliest due date; provides optimal solution to minimize maximum task tardiness
            sort_prio = ["due_date", "processing_time"]
        else:
            raise NotImplementedError(f"{criterion} is not a valid greedy priorization criterion")
        current_time = 0
        for i in range(1, len(tasks) + 1):
            tasks = tasks.sort_values(by=sort_prio)
            next_task_id = tasks[~tasks["done"]].index[0]
            tasks, current_time = self._execute_task(tasks, next_task_id, current_time, i)
        return tasks.sort_values(by="execution_order")


    def neighborhood(self):
        """
        creates neighborhood with swaps and neighbors that don't violate tabu lists
        """
        possible_swaps = list(combinations(range(len(self.current_best_solution)), 2))  # contains (0, 1) but not (1, 0) etc. doesn't contain (0, 0)
        possible_swaps = [swap for swap in possible_swaps if swap not in self.swap_tabu_list]

        neighborhood = []
        swaps = []
        for _ in range(self.neighborhood_sample_size * 5):  # max iters
            swap = possible_swaps.pop()
            swap0_val = self.current_best_solution[swap[0]]
            neighbor = self.current_best_solution.copy()
            neighbor[swap[0]] = neighbor[swap[1]]
            neighbor[swap[1]] = swap0_val
            if neighbor not in self.candidate_tabu_list: 
                neighborhood.append(neighbor)
                swaps.append(swap)
                if len(neighborhood) == self.neighborhood_sample_size:
                    break
        self._update_swap_tabu(new_swaps=swaps)
        self._update_candidate_tabu_list(new_candidates=neighborhood)
        return neighborhood
    
    

    def _update_candidate_tabu_list(self, new_candidates: List[List[int]]):
        # random.sample: without replacement
        to_free = max(0, (len(self.swap_tabu_list) + len(new_candidates)) - self.max_len_candidate_tabu_list)
        self.candidate_tabu_list = self.candidate_tabu_list[to_free:] + random.sample(population=new_candidates, k=min(len(new_candidates), self.max_len_candidate_tabu_list))  # fifo        


    def _update_swap_tabu(self, new_swaps: List[Tuple[int, int]]):
        # random.sample: without replacement
        to_free = max(0, (len(self.swap_tabu_list) + len(new_swaps)) - self.max_len_swap_tabu_list)
        self.swap_tabu_list = self.swap_tabu_list[to_free:] + random.sample(population=new_swaps, k=min(len(new_swaps), self.max_len_candidate_tabu_list))  # fifo
        


    def _execute_task(self, tasks: pd.DataFrame, task_id: int, current_time: int, order: int) -> tuple[pd.DataFrame, int]:
        task = tasks.loc[task_id]
        start_time = current_time
        completion_time = start_time + task["processing_time"]
        tardiness = max(0, completion_time - task["due_date"])

        tasks.loc[task_id, "done"] = True
        tasks.loc[task_id, "tardiness"] = tardiness
        tasks.loc[task_id, "execution_order"] = order
        tasks.loc[task_id, "completion_time"] = completion_time
        return tasks, completion_time
    

    def execute_tasks(self, tasks: pd.DataFrame, execution_order: List[int]):
        current_time = 0
        for i, task_id in enumerate(execution_order):
            tasks, current_time = self._execute_task(tasks=tasks, task_id=task_id, current_time=current_time, order=i)
        return tasks

    def _total_tardiness(self, execution_oder: List[int]) -> int:
        return self.execute_tasks(tasks=self.tasks, execution_order=execution_oder)["tardiness"].sum()

    def run(self):
        greedy_cost = self._total_tardiness(execution_oder=self.current_best_solution)
        current_best_cost = greedy_cost
        for _ in range(self.n_iters):
            neighbors = self.neighborhood()
            # tabu here
            for neighbor in neighbors:
                cost_neighbor = self._total_tardiness(execution_oder=neighbor) 
                if cost_neighbor < current_best_cost:
                    current_best_cost = cost_neighbor
                    self.current_best_solution = neighbor
        return greedy_cost, current_best_cost


greedy_costs = []
tabu_best_costs = []
for i in range(100):
    tabu_search = TabuSearch(n_iters=5, n_tasks=5, max_len_candidate_tabu_list=5, max_len_swap_tabu_list=5, neighborhood_sample_size=5, greedy_criterion="SPT")
    greedy_cost, tabu_best_cost = tabu_search.run()
    greedy_costs.append(greedy_cost)
    tabu_best_costs.append(tabu_best_cost)
df_result = pd.DataFrame({"cost greedy (SPT)": greedy_costs, "cost tabu": tabu_best_costs})
df_result["tabu_wins"] = df_result["cost greedy (SPT)"] > df_result["cost tabu"]
df_result[df_result["tabu_wins"]]

,cost greedy (SPT),cost tabu,tabu_wins
11,67,65,True
19,93,92,True
26,64,63,True
34,104,102,True
40,70,69,True
77,64,63,True
83,93,92,True
97,64,63,True
